In [2]:
# Import packages and modules
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df  = pd.read_csv("../Data/dataset_dk3619_preprocessed_v1.csv")

In [5]:
df.head()

,HourUTC,HourDK,DK3619Code,hour,day_of_week,weekday_name,quarter,month,year,day_of_year,...,lag_39,lag_40,lag_41,lag_42,lag_43,lag_44,lag_45,lag_46,lag_47,lag_48
0,2020-01-01T00:00:00,2020-01-01 01:00:00,R_R,1,2,Wednesday,1,1,2020,1,...,39.452003,37.029076,34.272778,30.867796,27.691839,24.262220,21.446964,20.692560,20.760229,21.076712
1,2020-01-01T00:00:00,2020-01-01 01:00:00,Q_QB,1,2,Wednesday,1,1,2020,1,...,13.973817,13.383460,12.440164,11.175482,9.128553,7.690116,7.325366,7.297887,7.436323,7.600429
2,2020-01-01T00:00:00,2020-01-01 01:00:00,Q_QA,1,2,Wednesday,1,1,2020,1,...,9.143302,8.965540,8.633377,6.069151,3.955089,3.400593,3.144497,3.085907,3.090541,3.118976
3,2020-01-01T00:00:00,2020-01-01 01:00:00,PR_PR,1,2,Wednesday,1,1,2020,1,...,1208.678223,1180.071045,1051.362549,880.705933,789.150513,721.568359,703.513916,712.612061,736.379395,784.044617
4,2020-01-01T00:00:00,2020-01-01 01:00:00,P_P,1,2,Wednesday,1,1,2020,1,...,53.242626,53.199566,53.141167,50.922737,46.150143,42.539543,41.061405,40.467297,40.414558,40.375416


In [6]:
print(df['DK3619Code'])
dkcode_list = list(set(df['DK3619Code'].to_list()))
dkcode_list.sort()
id2dkcode = enumerate(dkcode_list)
id2dkcode = dict(id2dkcode)

dkcode2id = {}
for i in id2dkcode.items():
    dkcode2id[i[1]] = i[0]
print(dict(dkcode2id))

### Mapping data to numberic
# df['DK3619Code'] = df['DK3619Code'].map(dkcode2id)

0            R_R
1           Q_QB
2           Q_QA
3          PR_PR
4            P_P
           ...  
1254843      E_E
1254844      F_F
1254845      I_I
1254846      H_H
1254847      A_A
Name: DK3619Code, Length: 1254848, dtype: object
{'A_A': 0, 'C_CA': 1, 'C_CC': 2, 'C_CE_CF_CD': 3, 'C_CG': 4, 'C_CH': 5, 'C_CI': 6, 'C_CJ': 7, 'C_CK': 8, 'C_CL': 9, 'C_CM_CB': 10, 'D_D': 11, 'E_E': 12, 'F_F': 13, 'G_G': 14, 'H_H': 15, 'I_I': 16, 'J_JA': 17, 'J_JB_JC': 18, 'K_K': 19, 'L_L': 20, 'M_MA': 21, 'M_MB': 22, 'M_MC': 23, 'N_N': 24, 'O_O': 25, 'PR_PR': 26, 'P_P': 27, 'Q_QA': 28, 'Q_QB': 29, 'R_R': 30, 'S_S': 31}


### Choose Feauture will use for Model

In [7]:
columns = ['HourUTC', 'HourDK', 'hour', 'day_of_week',
              'weekday_name', 'quarter', 'month', 'year', 'day_of_year',
              'day_of_month', 'week_of_year', 'season', 'holiday', 'Consumption_MWh']


In [8]:
df.columns

Index(['HourUTC', 'HourDK', 'DK3619Code', 'hour', 'day_of_week',
       'weekday_name', 'quarter', 'month', 'year', 'day_of_year',
       'day_of_month', 'week_of_year', 'season', 'holiday', 'DK36Code',
       'DK36Title', 'DK19Code', 'DK19Title', 'Consumption_MWh', 'lag_24',
       'lag_25', 'lag_26', 'lag_27', 'lag_28', 'lag_29', 'lag_30', 'lag_31',
       'lag_32', 'lag_33', 'lag_34', 'lag_35', 'lag_36', 'lag_37', 'lag_38',
       'lag_39', 'lag_40', 'lag_41', 'lag_42', 'lag_43', 'lag_44', 'lag_45',
       'lag_46', 'lag_47', 'lag_48'],
      dtype='object')

In [9]:
# print(df['weekday_name'])
# weekday_list = list(set(df['weekday_name'].to_list()))
# weekday_list.sort()
# id2day = enumerate(weekday_list)
# id2day = dict(id2day)

# day2id = {}
# for i in id2day.items():
#     day2id[i[1]] = i[0]
# print(dict(day2id))

### Mapping data to numberic
# df['weekday_name'] = df['weekday_name'].map(day2id)

In [10]:
weekday_mapping = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}
### Mapping data to numberic
df['weekday_name'] = df['weekday_name'].map(weekday_mapping)

In [11]:
# print(df['season'])
# season_list = list(set(df['season'].to_list()))
# season_list.sort()
# id2season = enumerate(season_list)
# id2season = dict(id2season)

# season2id = {}
# for i in id2season.items():
#     season2id[i[1]] = i[0]
# print(dict(season2id))

### Season mapping
# df['season'] = df['season'].map(season2id)


In [12]:

### Season mapping

seasons_mapping = {
    'Spring': 0,
    'Summer': 1,
    'Autumn': 2,
    'Winter': 3
}

df['season'] = df['season'].map(seasons_mapping)

In [13]:
df.head()

,HourUTC,HourDK,DK3619Code,hour,day_of_week,weekday_name,quarter,month,year,day_of_year,...,lag_39,lag_40,lag_41,lag_42,lag_43,lag_44,lag_45,lag_46,lag_47,lag_48
0,2020-01-01T00:00:00,2020-01-01 01:00:00,R_R,1,2,2,1,1,2020,1,...,39.452003,37.029076,34.272778,30.867796,27.691839,24.262220,21.446964,20.692560,20.760229,21.076712
1,2020-01-01T00:00:00,2020-01-01 01:00:00,Q_QB,1,2,2,1,1,2020,1,...,13.973817,13.383460,12.440164,11.175482,9.128553,7.690116,7.325366,7.297887,7.436323,7.600429
2,2020-01-01T00:00:00,2020-01-01 01:00:00,Q_QA,1,2,2,1,1,2020,1,...,9.143302,8.965540,8.633377,6.069151,3.955089,3.400593,3.144497,3.085907,3.090541,3.118976
3,2020-01-01T00:00:00,2020-01-01 01:00:00,PR_PR,1,2,2,1,1,2020,1,...,1208.678223,1180.071045,1051.362549,880.705933,789.150513,721.568359,703.513916,712.612061,736.379395,784.044617
4,2020-01-01T00:00:00,2020-01-01 01:00:00,P_P,1,2,2,1,1,2020,1,...,53.242626,53.199566,53.141167,50.922737,46.150143,42.539543,41.061405,40.467297,40.414558,40.375416


### Split data

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dense, Bidirectional
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import TimeDistributed, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

import os.path


In [15]:
# Create sequences
def create_sequences(df, seq_length):
    sequences = []
    labels = []
    for i in range(len(df) - seq_length):
        seq = df.iloc[i:i+seq_length].values
        label = df['Consumption_MWh'].iloc[i+seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [20]:
#all_dkcode_trainings
if not os.path.exists("../Checkpoints/"):
    os.makedirs("../Checkpoints/")

for i in dkcode_list:
    if os.path.exists("../Checkpoints/" + i + "_LSTMmodel.keras"):
        continue
    print('Training LSTM model with DKCode: ', i)
    df_filter = df[df['DK3619Code'] == i]
    df_corr = df_filter[columns]

    df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
    df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])

    df_corr = df_corr.drop(['HourUTC', 'HourDK'], axis=1)

    # Normalize the data
    scaler = MinMaxScaler()
    scaled_df = scaler.fit_transform(df_corr[['season', 'year', 'month', 'hour', 'Consumption_MWh']])

    scaled_df = pd.DataFrame(scaled_df, columns=['season', 'year', 'month', 'hour', 'Consumption_MWh'])

    #create sequence for training
    SEQ_LENGTH = 24  # for 24 hours sequence
    sequences, labels = create_sequences(scaled_df, SEQ_LENGTH)

    # Split the data into training (60%), validation (20%), and test (20%) sets
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, shuffle=False)

    #LSTM modeling
    model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

    # Model Training
    history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

    model.save("../Checkpoints/" + i + "_LSTMmodel.keras")
    model = tf.keras.models.load_model("../Checkpoints/" + i + "_LSTMmodel.keras")


    # Model Evaluation
    predictions = model.predict(X_test)

    # Calculate evaluation metrics
    r2 = r2_score(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)

    print(f'R² Score: {r2}')
    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')

Training LSTM model with DKCode:  F_F


<ipython-input-20-f5024330844a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
<ipython-input-20-f5024330844a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])


Epoch 1/10
1568/1568 [==============================] - 56s 33ms/step - loss: 0.0049 - val_loss: 0.0025 - lr: 0.0010
Epoch 2/10
1568/1568 [==============================] - 50s 32ms/step - loss: 0.0024 - val_loss: 0.0014 - lr: 0.0010
Epoch 3/10
1568/1568 [==============================] - 53s 34ms/step - loss: 0.0016 - val_loss: 9.9142e-04 - lr: 0.0010
Epoch 4/10
1568/1568 [==============================] - 48s 31ms/step - loss: 0.0012 - val_loss: 7.1255e-04 - lr: 0.0010
Epoch 5/10
1568/1568 [==============================] - 48s 31ms/step - loss: 9.8826e-04 - val_loss: 6.5189e-04 - lr: 0.0010
Epoch 6/10
1568/1568 [==============================] - 47s 30ms/step - loss: 9.2356e-04 - val_loss: 6.6573e-04 - lr: 0.0010
Epoch 7/10
1568/1568 [==============================] - 47s 30ms/step - loss: 8.2477e-04 - val_loss: 5.9964e-04 - lr: 0.0010
Epoch 8/10
1568/1568 [==============================] - 45s 28ms/step - loss: 7.8442e-04 - val_loss: 4.7184e-04 - lr: 0.0010
Epoch 9/10
1568/1568 [==

<ipython-input-20-f5024330844a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
<ipython-input-20-f5024330844a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])


Epoch 1/10
1568/1568 [==============================] - 50s 29ms/step - loss: 0.0085 - val_loss: 0.0023 - lr: 0.0010
Epoch 2/10
1568/1568 [==============================] - 44s 28ms/step - loss: 0.0029 - val_loss: 0.0012 - lr: 0.0010
Epoch 3/10
1568/1568 [==============================] - 47s 30ms/step - loss: 0.0018 - val_loss: 8.9480e-04 - lr: 0.0010
Epoch 4/10
1568/1568 [==============================] - 49s 31ms/step - loss: 0.0012 - val_loss: 0.0010 - lr: 0.0010
Epoch 5/10
1568/1568 [==============================] - 50s 32ms/step - loss: 9.6314e-04 - val_loss: 5.7039e-04 - lr: 0.0010
Epoch 6/10
1568/1568 [==============================] - 49s 31ms/step - loss: 8.2397e-04 - val_loss: 6.5338e-04 - lr: 0.0010
Epoch 7/10
1568/1568 [==============================] - 50s 32ms/step - loss: 6.9893e-04 - val_loss: 2.4352e-04 - lr: 0.0010
Epoch 8/10
1568/1568 [==============================] - 48s 30ms/step - loss: 6.3378e-04 - val_loss: 2.0157e-04 - lr: 0.0010
Epoch 9/10
1568/1568 [======

...

Epoch 1/10
1568/1568 [==============================] - 54s 32ms/step - loss: 0.0052 - val_loss: 0.0019 - lr: 0.0010
Epoch 2/10
1568/1568 [==============================] - 45s 29ms/step - loss: 0.0018 - val_loss: 7.3665e-04 - lr: 0.0010
Epoch 3/10
1568/1568 [==============================] - 44s 28ms/step - loss: 0.0011 - val_loss: 5.7060e-04 - lr: 0.0010
Epoch 4/10
1568/1568 [==============================] - 43s 27ms/step - loss: 8.1049e-04 - val_loss: 3.7305e-04 - lr: 0.0010
Epoch 5/10
1568/1568 [==============================] - 46s 29ms/step - loss: 6.4903e-04 - val_loss: 5.0636e-04 - lr: 0.0010
Epoch 6/10
1568/1568 [==============================] - 46s 30ms/step - loss: 5.7163e-04 - val_loss: 3.2412e-04 - lr: 0.0010
Epoch 7/10
1568/1568 [==============================] - 44s 28ms/step - loss: 4.8794e-04 - val_loss: 2.1975e-04 - lr: 0.0010
Epoch 8/10
1568/1568 [==============================] - 45s 29ms/step - loss: 4.3379e-04 - val_loss: 3.0769e-04 - lr: 0.0010
Epoch 9/10
1568/

<ipython-input-20-f5024330844a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
<ipython-input-20-f5024330844a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])


Epoch 1/10
1568/1568 [==============================] - 51s 29ms/step - loss: 0.0068 - val_loss: 0.0033 - lr: 0.0010
Epoch 2/10
1568/1568 [==============================] - 43s 27ms/step - loss: 0.0029 - val_loss: 0.0016 - lr: 0.0010
Epoch 3/10
1568/1568 [==============================] - 43s 27ms/step - loss: 0.0014 - val_loss: 5.2301e-04 - lr: 0.0010
Epoch 4/10
1568/1568 [==============================] - 42s 27ms/step - loss: 9.4681e-04 - val_loss: 4.1854e-04 - lr: 0.0010
Epoch 5/10
1568/1568 [==============================] - 43s 28ms/step - loss: 7.8443e-04 - val_loss: 4.3175e-04 - lr: 0.0010
Epoch 6/10
1568/1568 [==============================] - 44s 28ms/step - loss: 6.9546e-04 - val_loss: 3.4469e-04 - lr: 0.0010
Epoch 7/10
1568/1568 [==============================] - 45s 28ms/step - loss: 6.5603e-04 - val_loss: 6.3776e-04 - lr: 0.0010
Epoch 8/10
1568/1568 [==============================] - 51s 32ms/step - loss: 5.9400e-04 - val_loss: 2.5989e-04 - lr: 0.0010
Epoch 9/10
1568/1568

<ipython-input-20-f5024330844a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
<ipython-input-20-f5024330844a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])


Epoch 1/10
1568/1568 [==============================] - 53s 30ms/step - loss: 0.0038 - val_loss: 0.0011 - lr: 0.0010
Epoch 2/10
1568/1568 [==============================] - 45s 29ms/step - loss: 0.0013 - val_loss: 5.8736e-04 - lr: 0.0010
Epoch 3/10
1568/1568 [==============================] - 46s 29ms/step - loss: 8.1376e-04 - val_loss: 3.6813e-04 - lr: 0.0010
Epoch 4/10
1568/1568 [==============================] - 46s 29ms/step - loss: 6.1352e-04 - val_loss: 2.3836e-04 - lr: 0.0010
Epoch 5/10
1568/1568 [==============================] - 45s 29ms/step - loss: 5.2541e-04 - val_loss: 2.5120e-04 - lr: 0.0010
Epoch 6/10
1568/1568 [==============================] - 45s 29ms/step - loss: 4.5800e-04 - val_loss: 1.7141e-04 - lr: 0.0010
Epoch 7/10
1568/1568 [==============================] - 44s 28ms/step - loss: 4.4020e-04 - val_loss: 2.7003e-04 - lr: 0.0010
Epoch 8/10
1568/1568 [==============================] - 44s 28ms/step - loss: 4.1453e-04 - val_loss: 1.7213e-04 - lr: 0.0010
Epoch 9/10
1